<a href="https://colab.research.google.com/github/engbrunoconsultor/CNC_Machining/blob/feature_01/CNC_Machining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import requests, io, tempfile, os

In [16]:
owner = "engbrunoconsultor"
repo = "CNC_Machining"
branch = "feature_01"
root_paste = "data"




Descubra as keys que o arquivo possui

In [20]:
def descobre_keys(binario):
    with tempfile.NamedTemporaryFile(suffix=".h5", delete=False) as tmp:
        tmp.write(binario)
        path = tmp.name
    try:
        with pd.HDFStore(path, mode="r") as store:
            return store.keys()       # ex.: ['/df'] ou ['/table'] etc.
    finally:
        os.remove(path)

# exemplo só com o primeiro arquivo:
arq_teste = h5_file_list(owner, repo, branch,
                              "data/M01/OP00/good")[0]
binario   = requests.get(arq_teste["download_url"]).content
print(descobre_keys(binario))

[]


In [21]:
def bytes_to_df(h5_bytes, key=None):
    """Detecta a key se não for informada e devolve DataFrame."""
    import pandas as pd, tempfile, os

    with tempfile.NamedTemporaryFile(suffix=".h5", delete=False) as tmp:
        tmp.write(h5_bytes)
        path = tmp.name

    try:
        # Se o usuário NÃO passou key, tenta inferir:
        if key is None:
            try:
                return pd.read_hdf(path)           # pandas infere se houver 1 só
            except KeyError:
                pass                    # continua para detecção manual

        # Se key foi passada mas falhou ou ainda não sabemos:
        with pd.HDFStore(path, "r") as store:
            keys = store.keys()         # lista todas
        if not keys:
            raise ValueError("Arquivo HDF5 vazio")
        if key is None or key not in keys:
            key = keys[0]               # usa a primeira disponível
        return pd.read_hdf(path, key=key)
    finally:
        os.remove(path)


Ler o dataset certo e virar DataFrame

In [24]:
import pandas as pd, numpy as np, tempfile, os

def bytes_to_df_flex(h5_bytes, dataset_path=None):
    """
    Tenta ler via pandas; se falhar, abre com h5py
    e converte o dataset_path escolhido em DataFrame.
    """
    with tempfile.NamedTemporaryFile(suffix=".h5", delete=False) as tmp:
        tmp.write(h5_bytes)
        path = tmp.name

    try:
        # 1) primeiro tenta 'read_hdf' normal
        try:
            return pd.read_hdf(path)            # funciona se for estilo pandas
        except (ValueError, KeyError):
            pass                                # vai para fallback

        # 2) fallback h5py
        import h5py
        with h5py.File(path, "r") as f:
            # Se o usuário não informou qual dataset quer, pega o primeiro 2-D
            if dataset_path is None:
                # procura um dataset matriz (ndim==2)
                for name in f:
                    obj = f[name]
                    if isinstance(obj, h5py.Dataset) and obj.ndim == 2:
                        dataset_path = name
                        break
                else:
                    raise ValueError("Nenhum dataset 2-D encontrado.")

            data = f[dataset_path][()]          # lê como ndarray

        # cria DataFrame (colunas numeradas se não houver labels)
        if data.ndim == 1:
            df = pd.DataFrame({"value": data})
        else:
            cols = [f"col_{i}" for i in range(data.shape[1])]
            df = pd.DataFrame(data, columns=cols)

        return df

    finally:
        os.remove(path)


In [22]:
def h5_file_list(owner, repo, branch, api_path):
  url = f"https://api.github.com/repos/{owner}/{repo}/contents/{api_path}?ref={branch}"
  resp = requests.get(url)
  resp.raise_for_status()
  conteudo = resp.json()
  return [item for item in conteudo if item["name"].lower().endswith(".h5")]





Iterar sobre os itens

In [25]:
dfs = {}

for machine in ["M01"]:
    api_path = f"{root_paste}/{machine}/OP00/good"
    for arq in h5_file_list(owner, repo, branch, api_path):
        print(f"Lendo {arq['path']}…")
        binario = requests.get(arq["download_url"]).content   # baixa o .h5
        try:
          df = bytes_to_df_flex(binario)    # ← mudança aqui
          dfs[f"{machine}_{arq['name'][:-3]}"] = df
        except Exception as e:
          print(f" {arq['name']} ignorado: {e}")

print("Total lido =>", len(dfs), "DataFrames")

Lendo data/M01/OP00/good/M01_Aug_2019_OP00_000.h5…
Lendo data/M01/OP00/good/M01_Aug_2019_OP00_001.h5…
Lendo data/M01/OP00/good/M01_Aug_2019_OP00_003.h5…
Lendo data/M01/OP00/good/M01_Aug_2019_OP00_004.h5…
Lendo data/M01/OP00/good/M01_Aug_2019_OP00_005.h5…
Lendo data/M01/OP00/good/M01_Aug_2019_OP00_006.h5…
Lendo data/M01/OP00/good/M01_Aug_2019_OP00_007.h5…
Lendo data/M01/OP00/good/M01_Aug_2019_OP00_008.h5…
Lendo data/M01/OP00/good/M01_Aug_2019_OP00_009.h5…
Lendo data/M01/OP00/good/M01_Aug_2019_OP00_010.h5…
Lendo data/M01/OP00/good/M01_Aug_2019_OP00_011.h5…
Lendo data/M01/OP00/good/M01_Aug_2019_OP00_012.h5…
Lendo data/M01/OP00/good/M01_Aug_2019_OP00_013.h5…
Lendo data/M01/OP00/good/M01_Aug_2021_OP00_002.h5…
Lendo data/M01/OP00/good/M01_Aug_2021_OP00_003.h5…
Lendo data/M01/OP00/good/M01_Feb_2019_OP00_000.h5…
Lendo data/M01/OP00/good/M01_Feb_2019_OP00_001.h5…
Lendo data/M01/OP00/good/M01_Feb_2019_OP00_002.h5…
Lendo data/M01/OP00/good/M01_Feb_2019_OP00_003.h5…
Lendo data/M01/OP00/good/M01_Fe

Ver o que foi carregado

In [27]:
# Quantos e quais DataFrames temos?
print(f"{len(dfs)} dataframes no dicionário")
print(list(dfs.keys())[:10])        # mostra só os 10 primeiros nomes


29 dataframes no dicionário
['M01_M01_Aug_2019_OP00_000', 'M01_M01_Aug_2019_OP00_001', 'M01_M01_Aug_2019_OP00_003', 'M01_M01_Aug_2019_OP00_004', 'M01_M01_Aug_2019_OP00_005', 'M01_M01_Aug_2019_OP00_006', 'M01_M01_Aug_2019_OP00_007', 'M01_M01_Aug_2019_OP00_008', 'M01_M01_Aug_2019_OP00_009', 'M01_M01_Aug_2019_OP00_010']


 tamanho de cada um

In [28]:
for nome, df in dfs.items():
    print(f"{nome:<30}  linhas={len(df):>8}  colunas={df.shape[1]}")

M01_M01_Aug_2019_OP00_000       linhas=  268288  colunas=3
M01_M01_Aug_2019_OP00_001       linhas=  268288  colunas=3
M01_M01_Aug_2019_OP00_003       linhas=  268288  colunas=3
M01_M01_Aug_2019_OP00_004       linhas=  264192  colunas=3
M01_M01_Aug_2019_OP00_005       linhas=  268288  colunas=3
M01_M01_Aug_2019_OP00_006       linhas=  268288  colunas=3
M01_M01_Aug_2019_OP00_007       linhas=  269312  colunas=3
M01_M01_Aug_2019_OP00_008       linhas=  268288  colunas=3
M01_M01_Aug_2019_OP00_009       linhas=  263168  colunas=3
M01_M01_Aug_2019_OP00_010       linhas=  268288  colunas=3
M01_M01_Aug_2019_OP00_011       linhas=  268288  colunas=3
M01_M01_Aug_2019_OP00_012       linhas=  269312  colunas=3
M01_M01_Aug_2019_OP00_013       linhas=  268288  colunas=3
M01_M01_Aug_2021_OP00_002       linhas=  268288  colunas=3
M01_M01_Aug_2021_OP00_003       linhas=  269312  colunas=3
M01_M01_Feb_2019_OP00_000       linhas=  266400  colunas=3
M01_M01_Feb_2019_OP00_001       linhas=  265200  colunas

Abrir (exibir) um DataFrame específico

In [ ]:
df_m01 = dfs["M01_nomeDoArquivo"]   # troque pela key que apareceu acima
display(df_m01.head())              # mostra as 5 primeiras linhas
df_m01.info()                       # estrutura completa
df_m01.describe()                   # estatísticas básicas
